# Demo of Black-box checking of a Mealy machine

This is a demo of FalCAuN on Jupyter Notebook using the [kotlin-jupyter kernel](https://github.com/Kotlin/kotlin-jupyter). This demo assumes that `jupyter` is executed with the following environmental variables.

- `JAVA_HOME` (the java home for Java 17)

## Dependent packages and logger config

This notebook depends on FalCAuN-core.

In [1]:
@file:DependsOn("net.maswag.falcaun:FalCAuN-core:1.0-SNAPSHOT")

In [ ]:
import ch.qos.logback.classic.Level
import ch.qos.logback.classic.Logger
import de.learnlib.driver.simulator.MealySimulatorSUL
import de.learnlib.oracle.membership.SULOracle
import net.automatalib.alphabet.Alphabets
import net.automatalib.automaton.transducer.CompactMealy
import net.automatalib.modelchecker.ltsmin.AbstractLTSmin
import net.automatalib.modelchecker.ltsmin.LTSminVersion
import net.automatalib.util.automaton.builder.AutomatonBuilders
import net.automatalib.visualization.Visualization
import net.maswag.falcaun.*
import net.maswag.falcaun.parser.LTLFactory
import org.slf4j.LoggerFactory
import java.util.*

// The following surprises the debug log
var updaterLogger = LoggerFactory.getLogger(AbstractAdaptiveSTLUpdater::class.java) as Logger
updaterLogger.level = Level.INFO
var updateListLogger = LoggerFactory.getLogger(AdaptiveSTLList::class.java) as Logger
updateListLogger.level = Level.INFO
var LTSminVersionLogger = LoggerFactory.getLogger(LTSminVersion::class.java) as Logger
LTSminVersionLogger.level = Level.INFO
var AbstractLTSminLogger = LoggerFactory.getLogger(AbstractLTSmin::class.java) as Logger
AbstractLTSminLogger.level = Level.INFO
var EQSearchProblemLogger = LoggerFactory.getLogger(EQSearchProblem::class.java) as Logger
EQSearchProblemLogger.level = Level.INFO
var SimulinkSteadyStateGeneticAlgorithmLogger = LoggerFactory.getLogger(EQSteadyStateGeneticAlgorithm::class.java) as Logger
SimulinkSteadyStateGeneticAlgorithmLogger.level = Level.INFO

## The target Mealy machine

In [3]:
// Define the target Mealy machine
/// input alphabet contains strings "a" and "b"
val sigma = Alphabets.fromList(listOf("a", "b"))
/// output alphabet contains strings "p" and "q"
val gamma = Alphabets.fromList(listOf("p", "q"))
/// create Mealy machine
// @formatter:off
val target: CompactMealy<String, String> = AutomatonBuilders.newMealy<String, String>(sigma)
    .withInitial("q0")
    .from("q0")
        .on("a").withOutput("p").to("q1")
        .on("b").withOutput("q").to("q2")
    .from("q1")
        .on("a").withOutput("q").to("q0")
        .on("b").withOutput("p").to("q3")
    .from("q2")
        .on("a").withOutput("p").to("q3")
        .on("b").withOutput("p").to("q0")
    .from("q3")
        .on("a").withOutput("q").to("q2")
        .on("b").withOutput("q").to("q1")
    .create()
// @formatter:on

In [4]:
// This shows the target Mealy machine in a new window.
Visualization.visualize(target.transitionGraphView(sigma), false)

## The LTL properties

In [5]:
// Define LTL properties
val ltlFactory = LTLFactory()
val ltlList = listOf(
        "[] (output == p -> X (output == q))", // This does not hold
        "[] ((input == a && output == p && (X input == a)) -> (X output == q))", // This holds
).map { stlString ->
    ltlFactory.parse(stlString)
}.toList()
val signalLength = 10 // We believe that the traces of length 10 are enough to verify/falsify the properties
val properties = AdaptiveSTLList(ltlList, signalLength)

## The LTL properties

In [6]:
// Define the SUL and oracle
val sul = MealySimulatorSUL(target)
val oracle = SULOracle(sul)
properties.setMemOracle(oracle)

// Configure and run the verifier
val verifier = BlackBoxVerifier(oracle, sul, properties, sigma)
// Timeout must be set before adding equivalence testing
verifier.setTimeout(5 * 60) // 5 minutes
verifier.addRandomWordEQOracle(
        1, // The minimum length of the random word
        10, // The maximum length of the random word
        1000, // The maximum number of tests
        Random(),
        1
)
val result = verifier.run()

## Show the learned Mealy machine

In [7]:
// This shows the learned Mealy machine in a new window. The execution is blocked until the window is closed
verifier.visualizeLearnedMealy()

## Print the result

In [8]:
if (result) {
    println("All the properties are likely satisfied")
} else {
    println("Some properties are falsified")
    for (i in 0 until verifier.cexProperty.size) {
        println("${verifier.cexProperty[i]} is falsified by the following counterexample:")
        println("cex concrete input: ${verifier.cexInput[i]}")
        println("cex output: ${verifier.cexOutput[i]}")
    }
}

Some properties are falsified
[] ( ( output == "p"  ) -> ( X ( output == "q"  ) ) ) is falsified by the following counterexample:
cex concrete input: a b
cex output: p p
